# Demo - Adversarial Training with MNIST

In [1]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks
from torchattacks import PGD, FGSM

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load MNIST Data

In [3]:
mnist_train = dsets.MNIST(root='data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
batch_size = 128

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=batch_size,
                                         shuffle=False)

## 2. Define Model

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out

In [6]:
model = CNN().cuda()

In [7]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
pgd_attack = PGD(model, eps=0.3, alpha=2/255, iters=10)

## 3. Train Model

In [9]:
num_epochs = 5

In [10]:
for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = pgd_attack(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/5], lter [100/468], Loss: 0.5790
Epoch [1/5], lter [200/468], Loss: 0.3486
Epoch [1/5], lter [300/468], Loss: 0.3366
Epoch [1/5], lter [400/468], Loss: 0.1530
Epoch [2/5], lter [100/468], Loss: 0.1944
Epoch [2/5], lter [200/468], Loss: 0.1711
Epoch [2/5], lter [300/468], Loss: 0.1957
Epoch [2/5], lter [400/468], Loss: 0.0703
Epoch [3/5], lter [100/468], Loss: 0.1212
Epoch [3/5], lter [200/468], Loss: 0.1374
Epoch [3/5], lter [300/468], Loss: 0.1460
Epoch [3/5], lter [400/468], Loss: 0.0441
Epoch [4/5], lter [100/468], Loss: 0.0964
Epoch [4/5], lter [200/468], Loss: 0.1320
Epoch [4/5], lter [300/468], Loss: 0.1162
Epoch [4/5], lter [400/468], Loss: 0.0307
Epoch [5/5], lter [100/468], Loss: 0.0912
Epoch [5/5], lter [200/468], Loss: 0.1160
Epoch [5/5], lter [300/468], Loss: 0.0942
Epoch [5/5], lter [400/468], Loss: 0.0246


## 4. Test Model

In [11]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of Clean images: %f %%' % (100 * float(correct) / total))

Accuracy of Clean images: 99.050000 %


In [12]:
model.eval()

correct = 0
total = 0

fgsm_attack = FGSM(model)

for images, labels in test_loader:
    
    images = fgsm_attack(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of Adversarial images: %f %%' % (100 * float(correct) / total))

Accuracy of Adversarial images: 98.950000 %
